## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
import numpy as np
import random
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

veiculos = load_csv('veiculos.csv')

df_posicoes = load_csv('df_posicoes.csv')

In [3]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]

consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [4]:
def get_tipo_onibus(id_onibus):

    onibus = df_posicoes[df_posicoes['id_onibus']==id_onibus]

    return onibus['modelo'].unique()[0]

In [5]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [6]:
df_posicoes.head()

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo
0,648P-10,71429,2025-08-29T12:17:58Z,-23.654854,-46.760556,True,Básico
1,648P-10,71246,2025-08-29T12:17:47Z,-23.626433,-46.705596,False,Biarticulado
2,648P-10,71277,2025-08-29T12:17:37Z,-23.579963,-46.685155,False,Articulado (18m)
3,648P-10,71251,2025-08-29T12:17:20Z,-23.590813,-46.688640,False,Articulado (23m)
4,702C-10,81310,2025-08-29T12:17:29Z,-23.580592,-46.736508,True,Biarticulado


In [7]:
df_posicoes.dtypes

linha           object
id_onibus        int64
timestamp       object
lat            float64
lon            float64
is_eletrico       bool
modelo          object
dtype: object

In [8]:
df_posicoes.shape

(156531, 7)

In [9]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

In [10]:
df_posicoes['id_onibus'].nunique()

11307

In [11]:
data_frame = []

for id_onibus in df_posicoes['id_onibus'].unique():
    df_onibus = df_posicoes[df_posicoes['id_onibus'] == id_onibus].reset_index(drop=True)
    df_onibus.sort_values(by='timestamp', inplace=True)
    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['lon']
            x_depois = df_onibus.loc[i+1, 'lon']
            y_atual = row['lat']
            y_depois = df_onibus.loc[i+1, 'lat']
            momento_inicial = row['timestamp']
            momento_final = df_onibus.loc[i+1, 'timestamp']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            #reprojetar SIRGAS 2000
            dist = np.sqrt((x_depois - x_atual)**2 + (y_depois - y_atual)**2)
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

df_final = pd.DataFrame(data_frame)

In [12]:
df_final = df_final.merge(
    veiculos[['id_veiculo', 'eletrico']],
    left_on='id_onibus',
    right_on='id_veiculo',
    how='left').drop(columns='id_veiculo')

In [13]:
df_final['is_eletrico'] = [random.choice([True, False]) for i in range(len(df_final))]

In [14]:
dados_trips_layer = []

for onibus in df_final['id_onibus'].unique():

    df_onibus = df_final[df_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    for i, row in df_onibus.iterrows():
        coordinates.append(row['ponto_inicial'])
        stamp = datetime.fromisoformat(row['momento_inicial'].isoformat())
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']

    dados_trips_layer.append({
        'id_onibus': onibus,
        'emissao_co2': emissao_total,
        'eletrico': row['eletrico'],
        'coordinates' : coordinates,
        'timestamps' : timestamps
    })


df_trips = pd.DataFrame(dados_trips_layer)

In [15]:
def init_on_zero(timestamp_list)->list[float]:

    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts.total_seconds()) for ts in new_timestamps]

df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [16]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [17]:
eval(coord_t)

[[-46.7605565, -23.6548535],
 [-46.7600795, -23.65436375],
 [-46.757994, -23.65200325],
 [-46.75778675, -23.6517585],
 [-46.75778675, -23.6517585],
 [-46.75778675, -23.6517585],
 [-46.7542395, -23.649037],
 [-46.7526175, -23.649198],
 [-46.750743, -23.649341],
 [-46.749903, -23.6487995],
 [-46.7491555, -23.648463],
 [-46.7487645, -23.648301],
 [-46.74846225, -23.64817675]]

In [18]:
df_trips.loc[0, 'timestamps']

[0, 8, 63, 101, 101, 137, 204, 231, 276, 321, 361, 406, 435]

In [19]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [20]:
save_csv(df_final, 'df_final.csv')

save_csv(df_trips, 'df_trips.csv')

Base salva em data\df_final.csv
Base salva em data\df_trips.csv
